In [4]:
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
import os
import pandas as pd

In [6]:
load_dotenv()

graph = Neo4jGraph(
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASSWORD')
)

In [8]:
result = graph.query("""MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
      (p)-[:PART_OF]->(c:Category {categoryName:'Produce'})
RETURN cust.contactName as CustomerName,
       sum(o.quantity) AS TotalProductsPurchased
""")

print(result)

[{'CustomerName': 'Paula Wilson', 'TotalProductsPurchased': 77}, {'CustomerName': 'Art Braunschweiger', 'TotalProductsPurchased': 16}, {'CustomerName': 'Mary Saveley', 'TotalProductsPurchased': 45}, {'CustomerName': 'Victoria Ashworth', 'TotalProductsPurchased': 37}, {'CustomerName': 'Laurence Lebihan', 'TotalProductsPurchased': 118}, {'CustomerName': 'Palle Ibsen', 'TotalProductsPurchased': 70}, {'CustomerName': 'Jose Pavarotti', 'TotalProductsPurchased': 152}, {'CustomerName': 'Martine Rancé', 'TotalProductsPurchased': 50}, {'CustomerName': 'Horst Kloss', 'TotalProductsPurchased': 242}, {'CustomerName': 'Henriette Pfalzheim', 'TotalProductsPurchased': 85}, {'CustomerName': 'André Fonseca', 'TotalProductsPurchased': 31}, {'CustomerName': 'Daniel Tonini', 'TotalProductsPurchased': 5}, {'CustomerName': 'Maurizio Moroni', 'TotalProductsPurchased': 52}, {'CustomerName': 'Jonas Bergulfsen', 'TotalProductsPurchased': 18}, {'CustomerName': 'Yvonne Moncada', 'TotalProductsPurchased': 18}, {'C

In [9]:
print(graph.schema)

Node properties are the following:
Territory {territoryID: STRING, territoryDescription: STRING},Region {regionID: STRING, regionDescription: STRING},Supplier {supplierID: STRING, companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, phone: STRING, fax: STRING, homePage: STRING},Shipper {companyName: STRING, phone: STRING, shipperID: STRING},Product {productID: STRING, productName: STRING, unitPrice: FLOAT, unitsInStock: INTEGER, unitsOnOrder: INTEGER, reorderLevel: INTEGER, discontinued: BOOLEAN},Category {categoryID: STRING, categoryName: STRING, description: STRING},Customer {companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, phone: STRING, fax: STRING, customerID: STRING},Employee {address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, employeeID: STRING, lastName: 

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

embedding_provider = OpenAIEmbeddings(
    openai_api_key="sk-..."
)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://localhost:7687",
    username="neo4j",
    password="pleaseletmein",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

result = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.")
for doc in result:
    print(doc.metadata["title"], "-", doc.page_content)

ValueError: Could not connect to Neo4j database. Please ensure that the username and password are correct